<a href="https://colab.research.google.com/github/robitussin/CCDEPLRL_EXERCISES/blob/main/Exercise8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 8

Instructions: Finetune a model to answer the questions below.

In [2]:
!pip install --upgrade transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.2
    Uninstalling transformers-4.52.2:
      Successfully uninstalled transformers-4.52.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcs

In [3]:
!git config --global user.email "christianbongao@gmail.com"
!git config --global user.name "kamatis05"

In [29]:
from datasets import load_dataset

squad = load_dataset("squad", split="train[:5000]")

## A. Model finetuning

In [30]:
squad = squad.train_test_split(test_size=0.2)

In [31]:
squad["train"][0]
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'
}

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [32]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [42]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        stride=128,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]
        answer = examples["answers"][sample_index]

        # Skip examples without an answer
        if len(answer["answer_start"]) == 0 or len(answer["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            continue

        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # Find start and end token indices in the context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # Check if the answer is in the context span
        if offsets[context_start][0] > end_char or offsets[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Find start token index
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            # Find end token index
            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [43]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [54]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [55]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [78]:

from transformers import create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)


In [79]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [80]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [81]:
from huggingface_hub import notebook_login
notebook_login()

In [82]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="my_awesome_qa_model",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/my_awesome_qa_model is already a clone of https://huggingface.co/machiwao/my_awesome_qa_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [86]:
!pip install --upgrade transformers


In [89]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])


Epoch 1/3
255/255 [==============================] - ETA: 0s - loss: 1.4245

Several commits (8) will be pushed upstream.


255/255 [==============================] - 254s 937ms/step - loss: 1.4245 - val_loss: 1.5005
Epoch 2/3
255/255 [==============================] - ETA: 0s - loss: 0.9623

Several commits (9) will be pushed upstream.


255/255 [==============================] - 229s 899ms/step - loss: 0.9623 - val_loss: 1.5254
Epoch 3/3
255/255 [==============================] - ETA: 0s - loss: 0.8425

Several commits (10) will be pushed upstream.


255/255 [==============================] - 206s 808ms/step - loss: 0.8425 - val_loss: 1.5254


## B. Question Set

1. Question 1

In [90]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="my_awesome_qa_model")

Device set to use cuda:0


In [117]:
context1 = "The Pacific Ocean is the largest and deepest of Earth's oceanic divisions. It extends from the Arctic Ocean in the north to the Southern Ocean in the south"
question1 = "What is the largest ocean on Earth?"
# Expected Answer: Pacific Ocean

question_answerer(question=question1, context=context1)

{'score': 0.11851657181978226,
 'start': 4,
 'end': 17,
 'answer': 'Pacific Ocean'}

2. Question 2

In [126]:
context2 = "Isaac Newton was an English Mathematician, Physicist, Astronomer, and author who is widely recognised as one of the greatest mathematicians and physicists of all time"
question2 = "What was Isaac Newton's nationality?"
# Expected Answer: English

question_answerer(question=question2, context=context2)

{'score': 0.2555777132511139, 'start': 20, 'end': 27, 'answer': 'English'}

3. Question 3

In [120]:
context3 ="Mount Everest is Earth's highest mountain above sea level, located in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China."
question3 = "where is Mount Everest located?"
# Expected Answer: in the Himalayas on the border between Nepal and the Tibet Autonomous Region of China

question_answerer(question=question3, context=context3)

{'score': 0.20591996610164642,
 'start': 17,
 'end': 83,
 'answer': "Earth's highest mountain above sea level, located in the Himalayas"}

4. Question 4

In [129]:
context4 = "Photosynthesis is the process by which green plants and some other organisms use sunlight to synthesize foods from carbon dioxide and water"
Question4 = "What do plants use to perform photosynthesis?"
# Expected Answer: Sunlight

question_answerer(question=Question4, context=context4)

{'score': 0.13574345409870148, 'start': 81, 'end': 89, 'answer': 'sunlight'}

5. Question 5

In [130]:
context5 = "Barack Obama served as the 44th president of the United States from 2009 to 2017."
question5 = "When did Barack Obama serve as president?"
# Expected Answer: from 2009 to 2017

question_answerer(question=question5, context=context5)

{'score': 0.2622985541820526, 'start': 68, 'end': 80, 'answer': '2009 to 2017'}